In [ ]:
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
import zipfile
import os

zip_path = "yale face database.zip"
extract_path = "unzipped_folder"


if os.path.exists(zip_path):
    try:
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(extract_path)
        print("Files extracted to:", extract_path)
        print("Contents:", os.listdir(extract_path))
    except zipfile.BadZipFile:
        print(" Error: The file is not a valid zip or is corrupted.")
else:
    print("Error: Zip file not found. Available files are:", os.listdir())


Files extracted to: unzipped_folder
Contents: ['subject09.normal', 'subject01.glasses', 'subject05.wink', 'subject11.surprised', 'subject12.sleepy', 'subject07.centerlight', 'subject03.glasses', 'subject14.wink', 'subject10.wink', 'subject07.wink', 'subject02.happy', 'subject03.rightlight', 'subject09.happy', 'subject03.normal', 'subject10.normal', 'subject13.sad', 'subject11.sad', 'subject03.leftlight', 'subject06.happy', 'subject08.wink', 'subject14.leftlight', 'subject06.leftlight', 'subject15.happy', 'subject08.centerlight', 'subject12.happy', 'subject13.surprised', 'subject07.leftlight', 'subject14.centerlight', 'subject05.noglasses', 'subject02.leftlight', 'subject04.normal', 'subject13.sleepy', 'subject04.surprised', 'data', 'subject02.surprised', 'subject07.surprised', 'subject06.sleepy', 'subject14.noglasses', 'subject05.sad', 'subject06.wink', 'subject06.rightlight', 'subject01.rightlight', 'subject10.happy', 'subject08.leftlight', 'subject09.surprised', 'subject04.noglasses'

In [ ]:
images = []
labels = []


data_folder_path = os.path.join(extract_path, "data")

print(f"\nChecking for data folder at: {data_folder_path}")

if os.path.exists(data_folder_path) and os.path.isdir(data_folder_path):
    print("Data folder found. Loading images...")
    print("Files:", os.listdir(data_folder_path))

    for image_file in os.listdir(data_folder_path):
        image_path = os.path.join(data_folder_path, image_file)

        if os.path.isfile(image_path) and not image_file.startswith('.'):
            # Load the image in grayscale
            img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

            if img is not None:
                images.append(img)


                label = image_file.split('.')[0]
                labels.append(label)
else:
    print(f" Error: Could not find the 'data' folder at {data_folder_path}")

# Converison to NumPy Arrays
images = np.array(images)
labels = np.array(labels)

print(f"\nLoaded {len(images)} images with {len(np.unique(labels))} unique subjects.")
print("Subjects:", np.unique(labels))

# Flatten Images + Train/Test Split
X = images.reshape(len(images), -1)  # Flatten 2D image → 1D vector
y = labels

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#  Apply PCA for Dimensionality Reduction
pca = PCA(n_components=100, whiten=True, random_state=42)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

# Train Classifiers
# Classifier 1: Naive Bayes
class NaiveBayes:
    def fit(self, X, y):
        self.classes = np.unique(y)
        self.mean = {}
        self.var = {}
        self.prior = {}

        for c in self.classes:
            X_c = X[y == c]
            self.mean[c] = np.mean(X_c, axis=0)
            self.var[c] = np.var(X_c, axis=0)
            self.prior[c] = X_c.shape[0] / X.shape[0]

    def _calculate_probability(self, mean, var, x):
        # Add a small epsilon to variance to avoid division by zero or log(0)
        epsilon = 1e-9
        return np.exp(-(x - mean) ** 2 / (2 * (var + epsilon))) / np.sqrt(2 * np.pi * (var + epsilon))


    def _calculate_posterior(self, x):
        posteriors = []

        for c in self.classes:
            prior = np.log(self.prior[c])
            # Add a small epsilon to the probability before taking the log to avoid log(0)
            likelihood = np.sum(np.log(self._calculate_probability(self.mean[c], self.var[c], x) + 1e-9))
            posterior = prior + likelihood
            posteriors.append(posterior)

        return self.classes[np.argmax(posteriors)]

    def predict(self, X):
        return [self._calculate_posterior(x) for x in X]

# Training the classifier
nb = NaiveBayes()
nb.fit(X_train_pca, y_train) # Use X_train_pca and y_train

# Predicting
y_pred_nb = nb.predict(X_test_pca) # Use X_test_pca

# Accuracy
accuracy = accuracy_score(y_test, y_pred_nb) # Use y_test and y_pred_nb
print(f"Naive Bayes Accuracy: {accuracy:.2f}")


# Classifier 2: Logistic Regression
lr = LogisticRegression(max_iter=1000, random_state=42)
lr.fit(X_train_pca, y_train)
y_pred_lr = lr.predict(X_test_pca)

print("\n Logistic Regression Results:")
print("Accuracy:", accuracy_score(y_test, y_pred_lr))
print(classification_report(y_test, y_pred_lr))


Checking for data folder at: unzipped_folder/data
Data folder found. Loading images...
Files: ['subject09.normal', 'subject01.glasses', 'subject05.wink', 'subject11.surprised', 'subject12.sleepy', 'subject07.centerlight', 'subject03.glasses', 'subject14.wink', 'subject10.wink', 'subject07.wink', 'subject02.happy', 'subject03.rightlight', 'subject09.happy', 'subject03.normal', 'subject10.normal', 'subject13.sad', 'subject11.sad', 'subject03.leftlight', 'subject06.happy', 'subject08.wink', 'subject14.leftlight', 'subject06.leftlight', 'subject15.happy', 'subject08.centerlight', 'subject12.happy', 'subject13.surprised', 'subject07.leftlight', 'subject14.centerlight', 'subject05.noglasses', 'subject02.leftlight', 'subject04.normal', 'subject13.sleepy', 'subject04.surprised', 'subject02.surprised', 'subject07.surprised', 'subject06.sleepy', 'subject14.noglasses', 'subject05.sad', 'subject06.wink', 'subject06.rightlight', 'subject01.rightlight', 'subject10.happy', 'subject08.leftlight', 'su

In [ ]:
import matplotlib.pyplot as plt

# Show first 5 images
plt.figure(figsize=(10, 5))

for i in range(5):
    plt.subplot(1, 5, i+1)   # 1 row, 5 columns
    plt.imshow(images[i], cmap='gray')
    plt.title(labels[i])
    plt.axis("off")

plt.show()


In [ ]:
import matplotlib.pyplot as plt


acc_nb = accuracy_score(y_test, y_pred_nb)
acc_lr = accuracy_score(y_test, y_pred_lr)

# X-axis = classifiers, Y-axis = accuracy
classifiers = ["Naive Bayes", "Logistic Regression"]
accuracies = [acc_nb, acc_lr]

plt.figure(figsize=(6,4))
plt.plot(classifiers, accuracies, marker='o', linestyle='-', color='blue')
plt.title("Classifier Accuracy Comparison (Line Graph)")
plt.ylabel("Accuracy")
plt.ylim(0, 1)
plt.grid(True)
plt.show()
